* 并发
  - 并行：同时做某些事，可以互不干扰的同一时刻做几件事
  - 并发：也是同时做某些事，但是强调，一个时间内有事情要处理

In [ ]:
* threading

In [ ]:
import threading
import time


def worker():
    for i in range(5):
        time.sleep(1)
        print(i, threading.current_thread().name, 'working',
              threading.enumerate())


t1 = threading.Thread(target=worker, name='w1')
t2 = threading.Thread(target=worker, name='w2')
t1.start()
t2.start()

print('=====end=====', threading.enumerate())

In [ ]:
* daemon线程

In [ ]:
import threading
import time
import sys


def worker():
    c = threading.current_thread()
    for i in range(5):
        print(c.name, 'working', c.daemon)
        time.sleep(1)
        threading.Thread(target=lambda: print('+++'),
                         name='lambda').start()


for i in range(3):
    threading.Thread(target=worker, name='w{}'.format(i),
                     daemon=True).start()
else:
    # 还没start()时 当前线程时 主线程
    # daemon None 取默认值 => 当前线程的daemon=False
    threading.Thread(target=worker, name='x').start()

# False None ：non-daemon线程
# True : daemon线程
# t.start()
# time.sleep(2)
# print('=====end=====')
# 1/0  # 主线程异常没处理，导致后果会很严重
while True:
    time.sleep(1)
    print(threading.active_count(), threading.enumerate(),
          file=sys.stderr)
    if threading.active_count() <= 1:
        print('--------')
        break

# 主线程没事了，工作线程都是daemon线程，被迫全体结束
# 主线程没事了，抬起头，一看还有non-daemon线程，就什么也不做
# 当non-daemon结束了，主线程(进程)一看全是daemon线程
# daemon线程被迫全部结束
# ** 除主线程之外，non-daemon线程都结束了，如果主线程也没事干了，不管有没有daemon线程，进程结束；
# ** 除主线程之外，non-daemon线程都结束了，如果主线程还有事，进程继续

In [ ]:
* t.join()

In [ ]:
import threading
import time


def worker():
    c = threading.current_thread()
    for i in range(5):
        print(c.name, 'working', c.daemon)
        time.sleep(1)


t = threading.Thread(target=worker, name='w1',
                     daemon=True)
t.start()  # 创建线程就返回
print('-' * 30)
t.join()  # 当前线程(主线程)阻塞住了，直到t线程执行完毕
# join 加入谁？t，等t执行，join不写超时就是永久等，如果写，等timeout秒
# 谁调用，阻塞谁（阻塞当前线程得运行）

print('=' * 30)

# 线程同步

线程同步，线程间协同，通过某种技术，让一个线程访问某些数据时，其他线程不能访问这些数据，直到该线程完成对数据得操作

## Event 事件

Event事件， 是线程间通信机制中最简单的是实现，使用一个内部标记flag，通过flag的True或False的变化来进行操作

- set()    标记设置为True
- clear()  标记设置为False
- is_set() 标记是否为True
- wait(timeout=None)    设置等待标记为True的时长，None为无限等待。等到返回True，为等到超时了返回False

总结：
需要使用同一个Event对象的标记flag
谁wait就是登倒flag变为True，或登倒超时返回False
不限制等待者的个数，通知所有等待者
- 生产消费模型
- 订阅通知模式


## Lock 锁

In [ ]:
import threading
import time


# flag = False    # 没干完活
event = threading.Event()


def worker(e, count=10):
    cups = []
    # global flag
    # while not e.is_set():
    while not e.wait(0.5):
        print('I am working for U')
        if len(cups) >= count:
            # flag = True
            e.set()
            # break
            continue
        # time.sleep(0.5)  # 模拟生产1个杯子时间
        cups.append(1)
    print("Total cups done", len(cups))
    print('finished, cups={}'.format(len(cups)))


def boss(e):
    print('I am watching U')
    # while True:    # 轮询
    #     time.sleep(1)
    #     if flag:
    #         break
    # e.wait()    # 阻塞到状态为True
    while not e.wait(1):
        pass
    # 数据准备好了，下面可以做更多其他事情
    print('good job')


w = threading.Thread(target=worker, args=(event,), name='w')
b1 = threading.Thread(target=boss, args=(event,), name='b1')
b2 = threading.Thread(target=boss, args=(event,), name='b2')

b1.start()
b2.start()
w.start()

# n个观察者，变化后通知观察者
# 生产消费模型，消费者消费
# while True: pass 每隔多少时间问一次，如果没有完成，白问；浪费CPU时间片
# 通知，发布订阅模式 e.wait() # 阻塞 等一个状态的变化

In [ ]:
* logging

In [ ]:
RootLogger root

In [ ]:
import logging


# thread threadName process processName
# 调整root记录器的配置函数
# level级别，filename为root增加一个FileHandler
# 如果没有Filename，就会为root增加一个StreamHandler（console）
# basicConfig之后，root的handlers属性列表中只有一个handler
FORMAT = "[%(asctime)s] %(thread)s %(message)s"
logging.basicConfig(level=logging.INFO, format=FORMAT,
                    datefmt="%Y-%m-%d %H:%M:%S",
                    filename='D:/test.log',# 有了这个参数，就不去控制台，默认a模式
                    filemode='w'
                    )


# 日志记录，需要使用 日志记录器 对象，记录器可以控制输出到console或文件
# logging当中已经默认定义了一个root：根logger对象

# root = RootLogger(WARNING)
# logging.root  # 使用日志消息限制级别为警告WARNING，消息默认写到console(控制台)
# 日志级别：DEBUG INFO WARNING ERROR CRITICAL
# 消息：生成一个日志消息，消息有严重程度，消息有级别
# 日志记录器：也有级别 消息想通过改记录器记录下来，首先必须和记录器级别进行大小比较
# 日志级别 record's level >= 记录器级别 logger.level
# record's level >= logger的 **有效级别(logger)** 第一关

# 输出到控制台或文件 是日志记录器做的么？
# 不是记录器做的，是记录器中的处理器handler做的
# handler 是真正干活的，记录日志。不同类别：控制台handler(stdout\stderr)、文件handler
# logger可以拥有多个handler，有个属性handlers
# 每一个handler又有自己的level，只不过默认是0

# 控制输出格式：Formatter 每一个handler是输出者，它的格式就必须使用Formatter对象定义的格式输出


# logging.info('info message')  # 默认使用root_logger 记录器
# logging.warning('warning string')
# logging.error('error string')

print(logging.root.handlers)  # FileHandler h1
h2 = logging.StreamHandler()  # 没有设置format 走默认 %(message)s
h2.setFormatter(logging.Formatter('*** %(message)s ***'))
h2.setLevel(21)
logging.root.addHandler(h2)
print(logging.root.handlers)  # FileHandler h1

logging.root.handlers = []
logging.info('t100')  # root logger
print(logging.root.handlers)
print(logging.root)


In [ ]:
* 有效级别 effective level

- root      30
- a         级别0 有效级别30
- a.a1      级别0 有效级别30
- a.a1.a11  级别0 有效级别30

- root
- a1   # logger a1的父 不是a 而是root 注意
- a1.a12

logger = logging.getLogger('a.a1.a11')
logger.info('info test string')  # 不能输出  20>0 但是 <30 有效级别

有效级别定义：自己级别如果为0，取离自己最近不是0的祖先的级别为自己的有效级别

日志是否能输出，要看 记录的级别 是否 >= logger的有效级别 # 第一关

In [ ]:
import logging

logger = logging.getLogger('cmdb')   # 传播True
# logger.parent root
# logging.root.handlers = []
# logging.info('t100')  # root logger
# print(logging.root.handlers)
# print(logging.root)

# 自定义logger
# x = logging.getLogger()  # 无参数，默认获取是 root logger
# 继承和传播，有效level
# logger.Logger.manager 同一个字典中，sys.modules
# 为了简化，logger可以没有handler，但是默认propagate(传播)是打开的，向父传播
# 传播规则：root根，a b cmdb ... 的父 都是root
# a.a1 的父是a, a.a2的父是a # a.a1.a11  a11->a1->a->root
# 日志记录器初始化时 没有定义级别，默认时0 NOSET
# 这时候，第一关比较改为record's level >= logger effective level 有效级别
# 有效级别：离当前logger最近的哪个不是0的父logger(祖先logger)的级别
logger = logging.getLogger('cmdb')  # 某些单独使用日志配置
print(logger.handlers, logger.level, logger.getEffectiveLevel())
logger.info('5678')

In [ ]:
* 多线程中 输出最好用logging模块 方便 直观

In [ ]:
import threading
import logging


# flag = False    # 没干完活
event = threading.Event()
FORMAT = '%(asctime)s %(threadName)s %(thread)d %(levelname)s: %(message)s'
logging.basicConfig(level=logging.INFO, format=FORMAT, datefmt='%F %T')

def worker(e, count=10):
    cups = []
    # global flag
    # while not e.is_set():
    while not e.wait(0.5):
        logging.info('I am working for U')
        if len(cups) >= count:
            # flag = True
            e.set()
            # break
            continue
        # time.sleep(0.5)  # 模拟生产1个杯子时间
        cups.append(1)
    logging.info("Total cups done {}".format(len(cups)))
    logging.info('finished, cups={}'.format(len(cups)))


def boss(e):
    logging.info('I am watching U')
    # while True:    # 轮询
    #     time.sleep(1)
    #     if flag:
    #         break
    e.wait()    # 阻塞到状态为True
    # 数据准备好了，下面可以做更多其他事情
    logging.info('good job')


w = threading.Thread(target=worker, args=(event,), name='w')
b1 = threading.Thread(target=boss, args=(event,), name='b1')
b2 = threading.Thread(target=boss, args=(event,), name='b2')

b1.start()
b2.start()
w.start()

# Lock 
  - Lock类是mutex互斥锁
  - 一旦一个线程获得锁，其他**试图获取锁的线程**将被阻塞，只有拥有锁的线程释放锁
  - 凡是存在共享资源争抢的抵挡都可以使用锁，从而保证只有一个使用者可以完全使用这个资源
  
  - acquire(bloaking=True, timeout=-1)    
    - 默认阻塞，阻塞可以设置超时时间。非阻塞时，timeout禁止设置
  - release()   
    - 释放锁。可以从任何线程调用释放；
    - 已上锁的锁，会被重置为unlocked
    - 未上锁的上锁调用，抛RuntimeError异常